# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

We begin by importing the necessary packages. If the code cell below returns an error, please revisit the project instructions to double-check that you have installed Unity ML-Agents and NumPy.

In [1]:
from unityagents import UnityEnvironment
import numpy as np

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='Reacher.app')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='Reachers.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
# env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [ ]:
# !pip install regex

In [ ]:
# from ddpg_agent import Agent
from ddpg_agent import Agent
import os
import regex
from collections import deque
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def save_file(project_name, ext="pth"):
    file_number = []
    for file in os.listdir():
        if regex.match(r"{}".format("{}.*.{}".format(project_name, ext)), file):
            extract = regex.search(r'[0-9][0-9]', file)
            if file[extract.span()[0]] == "0":
                file_number.append(int(file[extract.span()[1]-1]))
            else:
                file_number.append(int("{}{}".format(file[extract.span()[0]],file[extract.span()[1]-1])))
            
            file_number.sort()
            file_number.reverse()
#             print(file_number)
            
            if file_number[0] < 9:
                file_name = "{}_0{}.{}".format(project_name, file_number[0]+1, ext)
            else:
                file_name = "{}_{}.{}".format(project_name, file_number[0]+1, ext)

    if len(file_number) == 0:
        file_name = "{}_00.{}".format(project_name, ext)
    
    print("Latest file name should be {}".format(file_name))
    return file_name

In [ ]:
seed = 1 # Seed for random control
agent = Agent(state_size, action_size, seed, num_agents)

In [ ]:
def ddpg(n_episodes=1200, max_t=1000):
    
    scores_deque = deque(maxlen=10)
    scores = []
    
    actor_pth = save_file("p2_cc_actor", ext="pth")
    critic_pth = save_file("p2_cc_critic", ext="pth")
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        agent.reset()
        score = np.zeros(num_agents)                           # initialize the score (for each agent)
        for t in range(max_t):
#         while True:
            actions = agent.act(states)                        # select an action (for each agent)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones, t)
            score += env_info.rewards                          # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        
        avg_scores = np.mean(score)
        scores_deque.append(avg_scores)
        scores.append(avg_scores)

        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(score)), end="")
        
        if i_episode % 10 == 0:
#             torch.save(agent.actor_local.state_dict(), actor_pth)
#             torch.save(agent.critic_local.state_dict(), critic_pth)
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))   
            
        # Early termination if the agent is training too slow
#         if i_episode == 30 and np.mean(scores_deque) < 1.:
#             print("The angent is learning too slow and terminating now")
#             return scores
        
        if i_episode == 200 and np.mean(scores_deque) < 1.:
            print("The angent is learning too slow and terminating now")
            return scores
    
    torch.save(agent.actor_local.state_dict(), actor_pth)
    torch.save(agent.critic_local.state_dict(), critic_pth)

    # Save the scores
    fo = open(save_file("p2_cc_score", ext = "txt"), "w")
    fo.writelines(str(scores))
    fo.close()
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.savefig('{}'.format(save_file("p2_cc_score_plt", ext="png")))
    plt.show()
    
    return scores


In [ ]:
scores = ddpg()

Latest file name should be p2_cc_actor_00.pth
Latest file name should be p2_cc_critic_00.pth
Episode 10	Average Score: 0.30	Score: 0.22
Episode 20	Average Score: 0.32	Score: 0.42
Episode 30	Average Score: 0.37	Score: 0.34
Episode 40	Average Score: 0.36	Score: 0.39
Episode 50	Average Score: 0.35	Score: 0.30
Episode 60	Average Score: 0.36	Score: 0.30
Episode 70	Average Score: 0.31	Score: 0.35
Episode 80	Average Score: 0.33	Score: 0.39
Episode 90	Average Score: 0.40	Score: 0.37
Episode 100	Average Score: 0.30	Score: 0.30
Episode 110	Average Score: 0.44	Score: 0.53
Episode 120	Average Score: 0.30	Score: 0.27
Episode 123	Average Score: 0.29	Score: 0.41